In [1]:
import pandas as pd
import numpy as np
import datetime
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF
from math import sqrt

In [6]:
# READ IN OBSERVED WINDS
obs = pd.read_csv('obs_wind.csv')   
obs['datetime'] = pd.to_datetime(obs['date'], format='%m/%d/%Y') 
obs.head()

,date,Observed,datetime
0,1/1/2016,NaN,2016-01-01
1,1/2/2016,NaN,2016-01-02
2,1/3/2016,NaN,2016-01-03
3,1/4/2016,NaN,2016-01-04
4,1/5/2016,NaN,2016-01-05


In [7]:
# READ IN REANALYSIS WINDS
era = pd.read_csv('ERA5-wind.csv')  
era['datetime'] = pd.to_datetime(era['datetime'], format='%m/%d/%Y')  # convert to datetime format
era['Month'] = era['datetime'].dt.month
era.head()

,datetime,reanalysis,Month
0,2016-01-01,0.967657,1
1,2016-01-02,0.951265,1
2,2016-01-03,1.021964,1
3,2016-01-04,1.164176,1
4,2016-01-05,1.027381,1


In [8]:
# MERGE OBSERVED AND REANALYSIS DATAFRAMES
merged = pd.merge(obs[['datetime','Observed']],era[['datetime','reanalysis']],how='inner',left_on='datetime', right_on='datetime')   
merged = merged.dropna(axis=0, how='any')
merged['Month'] = merged['datetime'].dt.month  
merged = merged.reset_index()
merged.head()

,index,datetime,Observed,reanalysis,Month
0,91,2016-04-01,2.27,0.922008,4
1,92,2016-04-02,2.70,0.956819,4
2,93,2016-04-03,2.22,1.342477,4
3,94,2016-04-04,2.17,1.274889,4
4,95,2016-04-05,2.22,1.128006,4


In [9]:
# CALCULATE AND APPLY QUANTILE MAPPING, NOTE: Different quantile map for summer and winter half-years
mod_ecdf_w = ECDF(merged['reanalysis'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)])  # winter half-year
mod_ecdf_s = ECDF(merged['reanalysis'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)])   # summer half-year

for j in range(len(era.datetime)):
    if (era.at[j,'Month'] <=3) or (era.at[j,'Month'] >= 10):   # winter half-year
        p = mod_ecdf_w(era.at[j,'reanalysis']) * 100   
        corr = np.percentile(merged['Observed'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)], p) - np.percentile(
            merged['reanalysis'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)], p)      
    else:
        p = mod_ecdf_s(era.at[j,'reanalysis']) * 100      # summer half-year
        corr = np.percentile(merged['Observed'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)], p) - np.percentile(
            merged['reanalysis'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)], p)
    era.at[j,'qmap_wind'] = era.at[j,'reanalysis'] + corr
    
era.head()

,datetime,reanalysis,Month,qmap_wind
0,2016-01-01,0.967657,1,2.068000
1,2016-01-02,0.951265,1,2.024384
2,2016-01-03,1.021964,1,2.118697
3,2016-01-04,1.164176,1,2.651586
4,2016-01-05,1.027381,1,2.116819


In [40]:
# PRINT OUT QUANTILE MAPPED VALUES
cols = ["qmap_wind"]
era.to_csv('qmap_wind.txt', sep=' ', float_format = "%.4f", 
           columns=cols, header=False, index=False)

In [35]:
# APPLY QUANTILE MAPPING TO HOURLY VALUES BY SCALING EACH HOURLY VALUE TO PRODUCE QUANTILE-MAPPED DAILY VALUE
era_hourly = pd.read_csv('wind-hourly.txt',header=None)
era_hourly.columns = ['hourly']
era_hourly['datetime'] = pd.date_range(start='1/1/2016',periods = 17544, freq='H')
era_hourly['Month'] = era_hourly['datetime'].dt.month  
era_hourly['Year'] = era_hourly['datetime'].dt.year
era_hourly['Day'] = era_hourly['datetime'].dt.day
era_hourly['Hour'] = era_hourly['datetime'].dt.hour
era['Year'] = era['datetime'].dt.year
era['Day'] = era['datetime'].dt.day
hourly_merged = pd.merge(era_hourly[['Month','Day','Year','Hour','hourly']],era[['Month','Day','Year','reanalysis','qmap_wind']],how='inner',left_on=('Month','Day','Year'), right_on=('Month','Day','Year'))   
hourly_merged['qmap_hourly'] = hourly_merged.hourly*hourly_merged.qmap_wind/hourly_merged.reanalysis
#hourly_merged
# PRINT OUT QUANTILE-MAPPED HOURLY WINDS
cols = ["Year","Month","Day","Hour","qmap_hourly"]
hourly_merged.to_csv('qmap_hourly.txt', sep=' ', float_format = "%.4f", 
           columns=cols, header=False, index=False)